<img src="https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width="200" align="left"><img src="coursera.png" width="100" align="right"><font size="6.5"><h1 align="center">Applied Data Science Capstone</font>

<h3 align="center"><font size="5">This notebook is intended for the final course in IBM Data Science Professional Certificate.</font>
<hr style="border: dashed rgb(0,0,0) 1.0px;background-color: rgb(0,0,255);height: 3.0px;"/>

## Week 3 - Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

### Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

<a href="#item0"> 0- Add-ons Part - Publish on GitHub</a>

<a href="#item1"> 1- Part (1) - Build code to scrape Wikipedia page</a>

<a href="#item2"> 2- Part (2) - Build code to get the latitude and the longitude</a>

<a href="#item3"> 3- Part (3) - Explore and cluster the neighborhoods</a> 
</font>
</div>

<hr style="border: dashed rgb(0,0,0) 1.0px;background-color: rgb(0,0,0);height: 1.0px;"/>

<a id="item0"></a>
<h2 id="item0">0- Add-ons Part - Publish on GitHub</h2>

In [11]:
# Clear the output
from IPython.display import clear_output

# Check Github
try:
    from github import Github
    print ('pygithub available')
except ImportError:
    # installing folium
    print ('Installing pygithub')
    !conda install -c conda-forge pygithub
    clear_output()
    print ('pygithub Installed')

pygithub available


In [13]:
# Import Required Libraries
import os    
import sys  
import base64
from github import Github
from github import InputGitTreeElement

In [14]:
# Read credentials from file
GitHubFile = open("GitHub.txt", "r")

for GitHublines in GitHubFile:
    if GitHublines.find("USER") != -1:
        GitUSER = str(GitHublines.replace("USER = ","")).rstrip()
    elif GitHublines.find("PASSWORD") != -1:
        GitPass = str(GitHublines.replace("PASSWORD = ","")).rstrip()
    elif GitHublines.find("REPO") != -1:
        GitRepo = str(GitHublines.replace("REPO = ","")).rstrip()

GitHubFile.close()

#### Create a function to publish on Github

In [15]:
def publishGitHub(filelist,filenames,msg):
    #Connect to Github
    g = Github(GitUSER,GitPass)
    repo = g.get_user().get_repo(GitRepo)
     
    # File list to publish and publish names
    file_list = filelist
    file_names = filenames
    commit_message = msg
    master_ref = repo.get_git_ref('heads/master')
    master_sha = master_ref.object.sha
    base_tree = repo.get_git_tree(master_sha)
    element_list = list()
    # Loop in list
    for i, entry in enumerate(file_list):
        with open(entry) as input_file:
            data = input_file.read()
        if entry.endswith('.png'):
            data = base64.b64encode(data)
        element = InputGitTreeElement(file_names[i], '100644', 'blob', data)
        element_list.append(element)
    tree = repo.create_git_tree(element_list, base_tree)
    parent = repo.get_git_commit(master_sha)
    commit = repo.create_git_commit(commit_message, tree, [parent])
    master_ref.edit(commit.sha)

<hr style="border: dashed rgb(0,0,0) 1.0px;background-color: rgb(0,0,0);height: 1.0px;"/>

<a id="item1"></a>
<h2 id="item1">1. Part (1) - Build code to scrape Wikipedia page, obtain the data in the table of postal codes and to transform the data into a pandas dataframe</h2>

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.
Start by creating a new Notebook for this assignment.
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [16]:
# import the library we use to open URLs
import urllib.request

# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup

# import pandas
import pandas as pd

In [17]:
# specify which URL/web page we are going to be scraping
url_wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# open the url using urllib.request and put the HTML into the page variable
page_wiki = urllib.request.urlopen(url_wiki)
# parse the HTML from our URL into the BeautifulSoup parse tree format
bs_wiki = BeautifulSoup(page_wiki, "lxml")

# use the 'find_all' function to bring back all instances of the 'table' tag in the HTML and store in 'all_tables' variable
tables_wiki=bs_wiki.find_all("table")

In [18]:
# Get the Required table
table_Pcodes=bs_wiki.find('table', class_='wikitable sortable')

# Lists to hold data
PostalCode=[]
Borough=[]
Neighborhood=[]

# Loop & ignore Borough = 'Not assigned'
for row in table_Pcodes.findAll('tr'):
    cells=row.findAll('td')
    if len(cells) > 0 :
        # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
        if cells[1].text.strip() != 'Not assigned':
            PostalCode.append(cells[0].text.strip())
            Borough.append(cells[1].text.strip())
            # If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
            if cells[2].text.strip() == 'Not assigned':
                Neighborhood.append(cells[1].text.strip())
            else:
                Neighborhood.append(cells[2].text.strip())

# Create dataframe to hold data        
df_wiki=pd.DataFrame(PostalCode,columns=['PostalCode'])
df_wiki['Borough']=Borough
df_wiki['Neighborhood']=Neighborhood

# show data shape
print (df_wiki.shape)

(211, 3)


In [19]:
# Group Values based on the Postal Code
df_Pcodes = df_wiki.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_Pcodes = df_Pcodes.sort_values(by ='Neighborhood' )
df_Pcodes.head(10)

,PostalCode,Borough,Neighborhood
58,M5H,Downtown Toronto,"Adelaide, King, Richmond"
12,M1S,Scarborough,Agincourt
14,M1V,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
89,M8W,Etobicoke,"Alderwood, Long Branch"
28,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights"
19,M2K,North York,Bayview Village
62,M5M,North York,"Bedford Park, Lawrence Manor East"
56,M5E,Downtown Toronto,Berczy Park
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [20]:
# show data shape
print (df_Pcodes.shape)

(103, 3)


#### Display Results same as required in submission

In [ ]:
# create a new dataframe
column_names = ["PostalCode", "Borough", "Neighborhood"]
df_forDisplay = pd.DataFrame(columns=column_names)

# Displayed list in assignment
Pcodes_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]
for postcode in Pcodes_list:
    df_forDisplay = df_forDisplay.append(df_Pcodes[df_Pcodes["PostalCode"]==postcode], ignore_index=True)
    
df_forDisplay

In [ ]:
# Publish Part 1 on GitHub
filelist = ['CT-Week3-14July.ipynb']
filenames = ['CT-Week3-Part1.ipynb']
msg='Capstone Week(3) - Part(1)'
publishGitHub(filelist,filenames,msg)

<hr style="border: dashed rgb(0,0,0) 1.0px;background-color: rgb(0,0,0);height: 1.0px;"/>

<a id="item2"></a>
<h2 id="item2">2. Part (2) - Build code to get the latitude and the longitude coordinates of each neighborhood</h2>

#### Option (1) : Using ArcGIS Geocoder rather than Google

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. 

In [ ]:
# Check geocoder
try:
    import pgeocode
    print ('Postal Geocoder available')
except ImportError:
    # installing geocoder
    print ('Installing Postal Geocoder')
    !pip install geocoder
    clear_output()
    print ('Postal Geocoder Installed')

In [ ]:
import geocoder # convert an address into latitude and longitude values

In [ ]:
# Lists to hold data
Latitude=[]
Longitude=[]

# Loop & Get coord for postal codes using ArcGIS Geocoder
for i in range(0,len(df_Pcodes)):
    address = df_Pcodes['PostalCode'].iloc[i] + ', canada'
    g= geocoder.arcgis(address)
    Latitude.append(g.lat)
    Longitude.append(g.lng)
    
# Create new dataframe to hold coord
df_Coords=df_Pcodes.copy()
df_Coords['Latitude']=Latitude
df_Coords['Longitude']=Longitude

df_Coords.head(10)

In [ ]:
# show data shape
print (df_Coords.shape)

#### Alternative option (Note: Reading from file as geocoder failed)

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [ ]:
# Lets download the dataset (-q )
!wget -q -O Geospatial_Coordinates.csv https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
print('Data downloaded!')

# Load Data From CSV File  
df_csv = pd.read_csv('Geospatial_Coordinates.csv')
df_csv.head()

In [ ]:
# Get the Long & Lat from the csv file and append it to main dataframe
df_final = pd.merge(df_Pcodes, df_csv, how='inner', left_on = 'PostalCode', right_on = 'Postal Code')

df_final.drop(['Postal Code'], axis = 1,inplace=True)
df_final.head(10)

#### Display Results same as required in submission

In [ ]:
# create a new dataframe
column_names = ["PostalCode", "Borough", "Neighborhood"]
df_forDisplay = pd.DataFrame(columns=column_names)

# Displayed list in assignment
Pcodes_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]
for postcode in Pcodes_list:
    df_forDisplay = df_forDisplay.append(df_final[df_final["PostalCode"]==postcode], ignore_index=True,sort=False)
    
df_forDisplay

In [ ]:
# show data shape
print (df_final.shape)

In [ ]:
# Publish Part 2 on GitHub
filelist = ['CT-Week3-14July.ipynb']
filenames = ['CT-Week3-Part2.ipynb']
msg='Capstone Week(3) - Part(2)'
publishGitHub(filelist,filenames,msg)

<hr style="border: dashed rgb(0,0,0) 1.0px;background-color: rgb(0,0,0);height: 1.0px;"/>

<a id="item3"></a>
<h2 id="item3">3. Part (3) - Explore and cluster the neighborhoods in Toronto</h2>

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you. 
Just make sure:

1- to add enough Markdown cells to explain what you decided to do and to report any observations you make. 

2- to generate maps to visualize your neighborhoods and how they cluster together. 

In [ ]:
# Checking for Required Packages availability

# Check folium
try:
    import folium
    print ('folium available')
except ImportError:
    # installing folium
    print ('Installing folium')
    !conda install -c conda-forge folium=0.5.0 --yes
    !pip install folium
    clear_output()
    print ('folium Installed')

# Check Esri GIS
try:
    import GIS
    print ('Esri GIS available')
except ImportError:
    # installing Esri GIS
    print ('Installing Esri GIS')
    !conda install -c esri arcgis --yes
    clear_output()
    print ('Esri GIS Installed')

In [ ]:
# import required libraries
from IPython.display import display
from IPython.display import HTML
from IPython.display import IFrame

# import Esri
import arcgis
import widgetsnbextension
import ipywidgets
from arcgis.gis import GIS

# import folium
import folium

#### Create a function to search venues to all the neighborhoods

In [ ]:
# create a function to search venues to all the neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### function to sort the venues in descending order

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Display Toronto Map with neighborhoods

In [ ]:
# Get the Coord of Toronto
g= geocoder.arcgis('Toronto, Canada')

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[g.lat, g.lng], zoom_start=10)
# Save it as html
map_toronto.save('Toronto.html')

# Optional:: Display Map
# display(map_toronto)

In [ ]:
# Get the Coord of Toronto
g= geocoder.arcgis('Toronto, Canada')

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[g.lat, g.lng], zoom_start=10)

# add markers to the map
for lat, lng, label in zip(df_final['Latitude'], df_final['Longitude'], df_final['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

# Save it as html
map_toronto.save('Neighborhood_Toronto.html')

# Optional:: Display Map
# display(map_toronto)

#### Filter only boroughs that contain the word Toronto

In [ ]:
# Select subset of the data frmae
df_simplified=df_final.copy()
df_toronto = df_simplified[df_simplified['Borough'].str.contains('Toronto')]
df_toronto.shape

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[g.lat, g.lng], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

# Save it as html
map_toronto.save('Neighborhood_Toronto_Simplified.html')

# Optional:: Display Map
# display(map_toronto)

#### Display Resulted Maps

In [ ]:
%%html
<iframe src="Neighborhood_Toronto.html"width="475" height="350"></iframe><iframe src="Neighborhood_Toronto_Simplified.html" 
width="475" height="350"></iframe>

#### Start using Foursquare API to explore the neighborhoods

In [ ]:
# Read credentials from file
FourSqrFile = open("Foursquare.txt", "r")

for FourSqrlines in FourSqrFile:
    if FourSqrlines.find("CLIENT_ID") != -1:
        CLIENT_ID = str(FourSqrlines.replace("CLIENT_ID = ","")).rstrip()
    elif FourSqrlines.find("CLIENT_SECRET") != -1:
        CLIENT_SECRET = str(FourSqrlines.replace("CLIENT_SECRET = ","")).rstrip()
    elif FourSqrlines.find("VERSION") != -1:
        VERSION = str(FourSqrlines.replace("VERSION = ","")).rstrip()

FourSqrFile.close()

# Set Limit Value
LIMIT = 100

In [ ]:
# let's get the venues that are within a radius of 500 meters.
df_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

# check the size of the resulting dataframe
print(df_venues.shape)
df_venues.head()

# check how many venues were returned for each neighborhood
df_venues.groupby('Neighborhood').count()

# find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(df_venues['Venue Category'].unique())))

#### Analyze Each Neighborhood

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(df_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = df_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

# check the size of the resulting dataframe
print(toronto_onehot.shape)
toronto_onehot.head()

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Let's confirm the new size & print each neighborhood along with the top 20 most common venues

In [ ]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

#### Cluster Neighborhoods

In [ ]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

Let's create a new dataframe that includes the cluster as well as the top 20 venues for each neighborhood.

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

Finally, let's visualize the resulting clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []

# add markers to map      
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine Clusters

Cluster 1

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Cluster 2

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Cluster 3

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Cluster 4

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Cluster 5

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

#### Observation

xxxxxxxxxxxxxxxxxxxxxxxx

In [ ]:
# Publish Part 3 on GitHub
filelist = ['CT-Week3-14July.ipynb']
filenames = ['CT-Week3-Part3.ipynb']
msg='Capstone Week(3) - Part(3)'
publishGitHub(filelist,filenames,msg)

<hr>

Copyright &copy; Ahmed Reda 2019.